In [ ]:
!pip install scikit-surprise

In [ ]:
!pip install numpy==1.26.4
!pip install --no-cache-dir scikit-surprise

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/aqi_suggested_activities_updated.csv')

# Convert AQI category to numerical rating based on actual AQI ranges
# Here, we'll use the upper boundary of each AQI range to represent the AQI rating
aqi_rating_map = {
    'Good': 50,  # 0-50
    'Moderate': 100,  # 51-100
    'Unhealthy for Sensitive Groups': 150,  # 101-150
    'Unhealthy': 200,  # 151-200
    'Very Unhealthy': 300,  # 201-300
    'Hazardous': 500  # 301-500
}
df["Rating"] = df["AQI_Category"].map(aqi_rating_map)

# Remove the AQI_Category column as per your request
df.drop(columns=["AQI_Category"], inplace=True)

# Rename columns to fit Surprise format
df.rename(columns={
    "Age": "User",  # Age is now numerical, used as 'User'
    "Time_of_Day": "Item"  # Time_of_Day is now numerical, used as 'Item'
}, inplace=True)

# Define Surprise Reader (adjust rating scale if needed)
reader = Reader(rating_scale=(0, 500))  # Rating scale based on actual AQI values

# Load dataset into Surprise
data = Dataset.load_from_df(df[["User", "Item", "Rating"]], reader)

# Split data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD Model
model = SVD()
model.fit(trainset)

# Evaluate Model
predictions = model.test(testset)
print(f"RMSE: {accuracy.rmse(predictions)}")
print(f"MAE: {accuracy.mae(predictions)}")

# Function to Recommend the Best Activity
def recommend_activity(user_age, time_of_day, df):
    # Predict ratings for all activities
    activity_scores = {}
    for activity in df["Suggested_Activity"].unique():
        # Predict rating for given user_age and time_of_day
        pred = model.predict(user_age, time_of_day).est  # Predict rating based on user and item
        activity_scores[activity] = pred

    # Get the best-rated activity
    best_activity = max(activity_scores, key=activity_scores.get)
    return best_activity


RMSE: 222.1154
RMSE: 222.11539793539754
MAE:  135.5350
MAE: 135.535


In [ ]:
# Example1 Input
user_age = 30  # Example age (numerical)
time_of_day = 9  # Time of day (numerical, e.g., 9 AM -> morning, 14 -> afternoon)
aqi_level = 2  # AQI category (corresponding to "Moderate")

# Get Recommendation
recommended_activity = recommend_activity(user_age, time_of_day, df)
print(f"Recommended Activity for Age {user_age}, Time {time_of_day}, AQI {aqi_level}: {recommended_activity}")

Recommended Activity for Age 30, Time 9, AQI 2: Stay indoors


In [ ]:
# Example2 Input
user_age2 = 20  # Example age (numerical)
time_of_day2 = 12  # Time of day (numerical, e.g., 9 AM -> morning, 14 -> afternoon)
aqi_level2 = 200  # AQI category (corresponding to "Moderate")

# Get Recommendation
recommended_activity = recommend_activity(user_age2, time_of_day2, df)
print(f"Recommended Activity for Age {user_age2}, Time {time_of_day2}, AQI {aqi_level2}: {recommended_activity}")

Recommended Activity for Age 20, Time 12, AQI 200: Stay indoors


In [ ]:
# Example3 Input
user_age3 = 20  # Example age (numerical)
time_of_day3 = 17  # Time of day (numerical, e.g., 9 AM -> morning, 14 -> afternoon)
aqi_level3 = 450  # AQI category (corresponding to "Moderate")

# Get Recommendation
recommended_activity = recommend_activity(user_age3, time_of_day3, df)
print(f"Recommended Activity for Age {user_age3}, Time {time_of_day3}, AQI {aqi_level3}: {recommended_activity}")

Recommended Activity for Age 20, Time 17, AQI 450: Stay indoors


In [ ]:
# Example4 Input
user_age4 = 20  # Example age (numerical)
time_of_day4 = 7  # Time of day (numerical, e.g., 9 AM -> morning, 14 -> afternoon)
aqi_level4 = 1  # AQI category (corresponding to "Moderate")

# Get Recommendation
recommended_activity = recommend_activity(user_age4, time_of_day4, df)
print(f"Recommended Activity for Age {user_age4}, Time {time_of_day4}, AQI {aqi_level4}: {recommended_activity}")

Recommended Activity for Age 20, Time 7, AQI 1: Stay indoors
